In [2]:
!pip3 install jupyter_contrib_nbextensions jupyter_nbextensions_configurator
!pip3 install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
!jupyter contrib nbextension install --user
!jupyter nbextensions_configurator enable --user


     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     --------------------------------------- 0.0/23.5 MB 119.1 kB/s eta 0:03:17
     --------------------------------------- 0.0/23.5 MB 140.3 kB/s eta 0:02:47
     --------------------------------------- 0.1/23.5 MB 399.4 kB/s eta 0:00:59
      --------------------------------------- 0.4/23.5 MB 1.2 MB/s eta 0:00:20
     - -------------------------------------- 0.6/23.5 MB 1.7 MB/s eta 0:00:14
     - -------------------------------------- 1.0/23.5 MB 2.2 MB/s eta 0:00:11
     -- ------------------------------------- 1.2/23.5 MB 2.5 MB/s eta 0:00:09
     -- ------------------------------------- 1.4/23.5 MB 2.8 MB/s eta 0:00:08
     --- ------------------------------------ 1.8/23.5 MB 3.1 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
     ---------------------------------------- 0.1/23.4 MB 4.8 MB/s eta 0:00:05
     ---------------------------------------- 0.2/23.4 MB 2.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/23.4 MB 2.3 MB/s eta 0:00:11
      --------------------------------------- 0.4/23.4 MB 2.4 MB/s eta 0:00:10
      --------------------------------------- 0.6/23.4 MB 2.6 MB/s eta 0:00:09
     - -------------------------------------- 0.8/23.4 MB 3.2 MB/s eta 0:00:08
     - -------------------------------------- 1.1/23.4 MB 3.4 MB/s eta 0:00:07
     -- ------------------------------------- 1.4/23.4 MB 3.7 MB/s eta 0:00:06
     -- ------------------------------------- 1.6/23.4 MB 3.7 MB/s eta 0:00:06
     --- ------------------------------------ 1.9/23.4 MB 4.2 MB/s eta 0:00:06
     --- ------------------------------------ 2.1/23.4 MB 4.2 MB/s eta 0:00:06
     --- ------------------------------------ 2.2/23.4 MB 3


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\joeyk\AppData\Local\Programs\Python\Python311\Scripts\jupyter-contrib.EXE\__main__.py", line 7, in <module>
  File "C:\Users\joeyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\jupyter_core\application.py", line 285, in launch_instance
    return super().launch_instance(argv=argv, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\joeyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1044, in launch_instance
    app = cls.instance(**kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\joeyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\configurable.py", line 556, in instance
    ins

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="Defining the environment and choices made during the project" data-toc-modified-id="Defining the environment and choices made during the project-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defining the environment and choices made during the project</a></span></li><li><span><a href="#Global Navigation" data-toc-modified-id="Global Navigation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Global Navigation</a></span></li><li><span><a href="#Filtering" data-toc-modified-id="Filtering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filtering</a></span></li><li><span><a href="#Vision" data-toc-modified-id="Vision-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Vision</a></span></li><li><span><a href="#Local avoidance" data-toc-modified-id="Local avoidance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Local avoidance</a></span></li><li><span><a href="#Motion control" data-toc-modified-id="Motion control-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Motion control</a>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# Introduction

In the scope of our course, "Basics of Mobile Robotics," we undertook a comprehensive project that integrated the key concepts covered throughout the semester's exercise sessions. Our objective was to synthesize our understanding of Global Navigation, Local Navigation, Filtering, Motion Control, and Vision into a practical application. The project encompassed several essential goals:
- Environment Creation: We created a setting with a predetermined set of obstacles. The Thymio robot has to find its way around this environment on its own, avoiding obstacles without using sensors to detect them.


# Defining the environment and choices made during the project


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Gloval Navigation


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Filtering


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Vision


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Local avoidance


# Motion control
